In [ ]:
from itertools import combinations
import torch
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load and clean dataset functions
def load_data(filepath):
    if filepath.endswith('.csv'):
        return pd.read_csv(filepath)
    elif filepath.endswith('.xlsx'):
        return pd.read_excel(filepath)
    else:
        raise ValueError(f"Unsupported file format: {filepath}")

def clean_dataset(df):
    if 'title' not in df.columns or 'label' not in df.columns:
        raise ValueError("The dataframe must have 'title' and 'label' columns.")

    df = df.dropna(subset=['title', 'label'])  # Ensure no NaNs in title or label
    df = df[df['title'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]  # Ensure non-empty strings
    df = df[df['label'].apply(lambda x: isinstance(x, (int, np.integer)) or str(x).isdigit())]  # Ensure valid labels

    df['title'] = df['title'].astype(str)
    df['label'] = df['label'].astype(int)
    return df

# Dataset wrapper to handle input data
class DatasetWrapper(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def create_data_loader(texts, labels, tokenizer, max_len, batch_size):
    print(f"Texts to tokenize: {texts[:5]}")  # Print the first 5 texts for debugging
    print(f"Labels: {labels[:5]}")            # Print the corresponding labels

    # Filter out any potential empty strings in the texts
    valid_data = [(text, label) for text, label in zip(texts, labels) if text.strip()]

    # Separate valid texts and labels
    valid_texts, valid_labels = zip(*valid_data) if valid_data else ([], [])

    if len(valid_texts) == 0:
        raise ValueError("No valid texts found after filtering.")

    encoding = tokenizer(
        valid_texts,
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )

    dataset = DatasetWrapper(encoding, valid_labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training and evaluation functions
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, dataset_len):
    model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / dataset_len, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, dataset_len):
    model.eval()
    losses = []
    correct_predictions = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    return correct_predictions.double() / dataset_len, np.mean(losses), all_labels, all_preds

# Main function
def main():
    RANDOM_SEED = 42
    MAX_LEN = 128
    BATCH_SIZE = 16
    EPOCHS = 3
    TEST_SPLIT = 0.2
    PATIENCE = 3  # Early stopping patience
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'Using device: {device}')

    languages = ['hebrew', 'arabic', 'bangla', 'chinese', 'english', 'german', 'indonesian', 'romanian', 'turkish']
    file_paths = {
        'hebrew': '/hebrew.xlsx',
        'arabic': '/arabic.xlsx',
        'bangla': '/bangla.csv',
        'chinese': '/chinese.csv',
        'english': '/english.csv',
        'german': '/german.csv',
        'indonesian': '/indonesian.csv',
        'romanian': '/romanianL.xlsx',
        'turkish': '/turkish.csv'
    }

    results = []
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    # Iterate over combinations of 2 languages
    for lang_comb in combinations(languages, 2):
        print(f"Processing combination: {lang_comb}")

        # Load and clean datasets for the two languages
        df_lang1 = load_data(file_paths[lang_comb[0]])
        df_lang1 = clean_dataset(df_lang1)

        df_lang2 = load_data(file_paths[lang_comb[1]])
        df_lang2 = clean_dataset(df_lang2)

        # Check if datasets are empty after cleaning
        if df_lang1.empty:
            print(f"Warning: Dataset for {lang_comb[0]} is empty after cleaning. Skipping this combination.")
            continue
        if df_lang2.empty:
            print(f"Warning: Dataset for {lang_comb[1]} is empty after cleaning. Skipping this combination.")
            continue

        # Split each dataset into 80% train and 20% test
        if len(df_lang1) > 1 and len(df_lang2) > 1:  # Ensure there are enough samples to split
            df_train_lang1, df_test_lang1 = train_test_split(df_lang1, test_size=TEST_SPLIT, random_state=RANDOM_SEED)
            df_train_lang2, df_test_lang2 = train_test_split(df_lang2, test_size=TEST_SPLIT, random_state=RANDOM_SEED)
        else:
            print(f"Warning: Dataset too small for {lang_comb}. Skipping.")
            continue

        # Combine train datasets of the two languages
        #df_train_combined = pd.concat([df_train_lang1, df_train_lang2], ignore_index=True)

        # Combine train datasets of the two languages
        df_train_combined = pd.concat([df_train_lang1, df_train_lang2], ignore_index=True)

        # Create DataLoaders
        train_data_loader = create_data_loader(
            df_train_combined['title'].tolist(),
            df_train_combined['label'].tolist(),
            tokenizer,
            MAX_LEN,
            BATCH_SIZE
        )
        test_data_loader_lang1 = create_data_loader(
            df_test_lang1['title'].tolist(),
            df_test_lang1['label'].tolist(),
            tokenizer,
            MAX_LEN,
            BATCH_SIZE
        )
        test_data_loader_lang2 = create_data_loader(
            df_test_lang2['title'].tolist(),
            df_test_lang2['label'].tolist(),
            tokenizer,
            MAX_LEN,
            BATCH_SIZE
        )

        # Initialize model
        model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
        model = model.to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
        total_steps = len(train_data_loader) * EPOCHS
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
        loss_fn = torch.nn.CrossEntropyLoss().to(device)

        best_loss = float('inf')
        patience_counter = 0

        # Training loop
        for epoch in range(EPOCHS):
            print(f'Starting epoch {epoch + 1}/{EPOCHS}')
            train_acc, train_loss = train_epoch(
                model,
                train_data_loader,
                loss_fn,
                optimizer,
                device,
                scheduler,
                len(df_train_combined)
            )
            print(f'Train loss {train_loss}, accuracy {train_acc}')

        # Evaluate on the two test sets separately
        print(f'Evaluating on {lang_comb[0]} test set...')
        test_acc_lang1, test_loss_lang1, labels_lang1, preds_lang1 = eval_model(
            model,
            test_data_loader_lang1,
            loss_fn,
            device,
            len(df_test_lang1)
        )
        precision_lang1 = precision_score(labels_lang1, preds_lang1, average='weighted')
        recall_lang1 = recall_score(labels_lang1, preds_lang1, average='weighted')
        f1_lang1 = f1_score(labels_lang1, preds_lang1, average='weighted')

        print(f'Evaluating on {lang_comb[1]} test set...')
        test_acc_lang2, test_loss_lang2, labels_lang2, preds_lang2 = eval_model(
            model,
            test_data_loader_lang2,
            loss_fn,
            device,
            len(df_test_lang2)
        )
        precision_lang2 = precision_score(labels_lang2, preds_lang2, average='weighted')
        recall_lang2 = recall_score(labels_lang2, preds_lang2, average='weighted')
        f1_lang2 = f1_score(labels_lang2, preds_lang2, average='weighted')

        # Save results for the current combination
        results.append({
            'languages': lang_comb,
            'test_results_lang1': {
                'accuracy': test_acc_lang1.item(),
                'precision': precision_lang1,
                'recall': recall_lang1,
                'f1': f1_lang1
            },
            'test_results_lang2': {
                'accuracy': test_acc_lang2.item(),
                'precision': precision_lang2,
                'recall': recall_lang2,
                'f1': f1_lang2
            }
        })

    print(f"Final results: {results}")

if __name__ == "__main__":
    main()


Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Processing combination: ('hebrew', 'arabic')
Texts to tokenize: [' זקוקים להחלפת ברך? ניתוחי החלפת ברך בשיטה הטובה ביותר', '  "אף אחד לא רוצה להיראות כאילו עבר השתלת שיער" - השיטה שאתם צריכים להכיר', 'סקר ראשון אחר הבחירות: גנץ מתחזק, רה"מ נחלש', " שנתיים לאחר מותה הטרגי בגיל 23 - נחשף שיר חדש של תמר 'טאי' עמר", 'גאוני. צפו: כך נבנה השער הדרמטי של הנוער']
Labels: [1, 1, 0, 0, 1]
Texts to tokenize: [' אל תשקיעו במערכת חימום גדולה עד שתראו את ההמצאה המהפכנית הזו...', ' ארגז כלים לטיפול בכאבי גב תחתון באופן עצמאי', '  עינב בובליל: "זה המוצר שגמל אותי ממתוקים"', 'אלעל פתחה עמדות צ\'ק אין עצמי בנתב"ג', 'הטעות של בני 55+']
Labels: [1, 1, 1, 0, 1]
Texts to tokenize: ['بالفيديو – شكل ابنتي #أشرف_زكي و #روجينا حديث #الجمهور... كشفتا والدهما؟', 'الموت يفجع حجاج عبد العظيم مرتين في 4 أيام', 'تاكوبيل تعيد البطاطا، الأمور إثارة!', 'الصحة: مد فترة الحملة القومية الثانية للتطعيم ضد مرض شلل الأطفال حتى غد', 'إحباط محاولة تهريب 20 مليون قرص أمفيتامين مخدر مخبأة داخل شحنة فاكهة العنب.\n']
Labels: [1, 0,

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting epoch 1/3


...

Starting epoch 1/3


<ipython-input-8-0feaa53fd63d>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Train loss 0.42449577358857926, accuracy 0.8028802010341661
Starting epoch 2/3
Train loss 0.3143414980983808, accuracy 0.8644469144155028
Starting epoch 3/3
Train loss 0.2183651244195849, accuracy 0.9137872710578456
Evaluating on indonesian test set...
Evaluating on romanian test set...


<ipython-input-8-0feaa53fd63d>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Processing combination: ('indonesian', 'turkish')
Processing combination: ('romanian', 'turkish')
Final results: [{'languages': ('hebrew', 'arabic'), 'test_results_lang1': {'accuracy': 0.8293838862559242, 'precision': 0.829517616635037, 'recall': 0.8293838862559242, 'f1': 0.8293762215607874}, 'test_results_lang2': {'accuracy': 0.8655896607431341, 'precision': 0.8691414458247566, 'recall': 0.865589660743134, 'f1': 0.8650791697953828}}, {'languages': ('hebrew', 'bangla'), 'test_results_lang1': {'accuracy': 0.8341232227488152, 'precision': 0.8365070574902124, 'recall': 0.8341232227488151, 'f1': 0.8337871449938657}, 'test_results_lang2': {'accuracy': 0.7219917012448133, 'precision': 0.7126977788123341, 'recall': 0.7219917012448133, 'f1': 0.7134814133935211}}, {'languages': ('hebrew', 'chinese'), 'test_results_lang1': {'accuracy': 0.7914691943127963, 'precision': 0.7965670190788673, 'recall': 0.7914691943127962, 'f1': 0.7904805842984762}, 'test_results_lang2': {'accuracy': 0.968187274909963

In [ ]:
pip install google-cloud-storage


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load and clean dataset functions
def load_data(filepath):
    if filepath.endswith('.csv'):
        return pd.read_csv(filepath)
    elif filepath.endswith('.xlsx'):
        return pd.read_excel(filepath)
    else:
        raise ValueError(f"Unsupported file format: {filepath}")

def clean_dataset(df):
    if 'title' not in df.columns or 'label' not in df.columns:
        raise ValueError("The dataframe must have 'title' and 'label' columns.")
    df = df.dropna(subset=['title', 'label'])  # Ensure no NaNs in title or label
    df = df[df['title'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]  # Ensure non-empty strings
    df = df[df['label'].apply(lambda x: isinstance(x, (int, np.integer)) or str(x).isdigit())]  # Ensure valid labels

    df['title'] = df['title'].astype(str)
    df['label'] = df['label'].astype(int)
    return df

# Dataset wrapper to handle input data
class DatasetWrapper(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def create_data_loader(texts, labels, tokenizer, max_len, batch_size):
    encoding = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )
    dataset = DatasetWrapper(encoding, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training and evaluation functions
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, dataset_len):
    model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / dataset_len, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, dataset_len):
    model.eval()
    losses = []
    correct_predictions = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    return correct_predictions.double() / dataset_len, np.mean(losses), all_labels, all_preds

# Main function to train on all data (split Hebrew) and test on Hebrew
def main():
    RANDOM_SEED = 42
    MAX_LEN = 128
    BATCH_SIZE = 16
    EPOCHS = 3
    TEST_SPLIT = 0.2  # Split ratio for the Hebrew dataset
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'Using device: {device}')

    languages = ['hebrew', 'arabic', 'bangla', 'chinese', 'english', 'german', 'indonesian', 'romanian', 'turkish']
    file_paths = {
        'hebrew': '/hebrew.xlsx',
        'arabic': '/arabic.xlsx',
        'bangla': '/bangla.csv',
        'chinese': '/chinese.csv',
        'english': '/english.csv',
        'german': '/german.csv',
        'indonesian': '/indonesian.csv',
        'romanian': '/romanianL.xlsx',
        'turkish': '/turkish.csv'
    }

    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    # Load and clean the Hebrew dataset, then split it
    df_hebrew = load_data(file_paths['hebrew'])
    df_hebrew = clean_dataset(df_hebrew)
    df_train_hebrew, df_test_hebrew = train_test_split(df_hebrew, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

    # Combine all non-Hebrew datasets for training
    train_dfs = [df_train_hebrew]  # Start with Hebrew train split
    for lang in languages:
        if lang != 'hebrew':
            df = load_data(file_paths[lang])
            df = clean_dataset(df)
            train_dfs.append(df)

    # Combine all training data
    df_train_combined = pd.concat(train_dfs, ignore_index=True)

    # Create DataLoaders for training and testing
    train_data_loader = create_data_loader(
        df_train_combined['title'].tolist(),
        df_train_combined['label'].tolist(),
        tokenizer,
        MAX_LEN,
        BATCH_SIZE
    )

    test_data_loader_hebrew = create_data_loader(
        df_test_hebrew['title'].tolist(),
        df_test_hebrew['label'].tolist(),
        tokenizer,
        MAX_LEN,
        BATCH_SIZE
    )

    # Initialize the model
    model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
    model = model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    total_steps = len(train_data_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    loss_fn = torch.nn.CrossEntropyLoss().to(device)

    # Training loop
    for epoch in range(EPOCHS):
        print(f'Starting epoch {epoch + 1}/{EPOCHS}')
        train_acc, train_loss = train_epoch(
            model,
            train_data_loader,
            loss_fn,
            optimizer,
            device,
            scheduler,
            len(df_train_combined)
        )
        print(f'Train loss {train_loss}, accuracy {train_acc}')

    # Evaluate on the Hebrew test set
    print(f'Evaluating on Hebrew test set...')
    test_acc, test_loss, labels, preds = eval_model(
        model,
        test_data_loader_hebrew,
        loss_fn,
        device,
        len(df_test_hebrew)
    )
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    print(f'Test Accuracy: {test_acc}')
    print(f'Test Precision: {precision}')
    print(f'Test Recall: {recall}')
    print(f'Test F1: {f1}')

if __name__ == "__main__":
    main()


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoin

Starting epoch 1/3


<ipython-input-1-3262acde9b8e>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Train loss 0.1777877322611112, accuracy 0.9228796550955177
Starting epoch 2/3
Train loss 0.11068957795370421, accuracy 0.9549914517208058
Starting epoch 3/3
Train loss 0.0742959509393722, accuracy 0.9696845808865433
Evaluating on Hebrew test set...
Test Accuracy: 0.8341232227488152
Test Precision: 0.8341606945917059
Test Recall: 0.8341232227488151
Test F1: 0.8341232227488151


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from google.cloud import storage

# Function to load and clean dataset
def load_data(filepath):
    if filepath.endswith('.csv'):
        return pd.read_csv(filepath)
    elif filepath.endswith('.xlsx'):
        return pd.read_excel(filepath)
    else:
        raise ValueError(f"Unsupported file format: {filepath}")

def clean_dataset(df):
    if 'title' not in df.columns or 'label' not in df.columns:
        raise ValueError("The dataframe must have 'title' and 'label' columns.")
    df = df.dropna(subset=['title', 'label'])  # Ensure no NaNs in title or label
    df = df[df['title'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]  # Ensure non-empty strings
    df = df[df['label'].apply(lambda x: isinstance(x, (int, np.integer)) or str(x).isdigit())]  # Ensure valid labels

    df['title'] = df['title'].astype(str)
    df['label'] = df['label'].astype(int)
    return df

# Dataset wrapper to handle input data
class DatasetWrapper(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def create_data_loader(texts, labels, tokenizer, max_len, batch_size):
    encoding = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )
    dataset = DatasetWrapper(encoding, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training and evaluation functions
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, dataset_len):
    model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / dataset_len, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, dataset_len):
    model.eval()
    losses = []
    correct_predictions = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    return correct_predictions.double() / dataset_len, np.mean(losses), all_labels, all_preds

# Save model to Google Cloud Storage (GCS)
#def save_model_to_gcs(model, bucket_name, model_filename):
 #   local_model_path = f"/tmp/{model_filename}"
 #   torch.save(model.state_dict(), local_model_path)

  #  client = storage.Client()
  #  bucket = client.get_bucket(bucket_name)
  #  blob = bucket.blob(model_filename)
  #  blob.upload_from_filename(local_model_path)
  #  print(f"Model saved to GCS at gs://{bucket_name}/{model_filename}")

# Main function to train on 8 languages and test on Hebrew, save models
def main():
    RANDOM_SEED = 42
    MAX_LEN = 128
    BATCH_SIZE = 16
    EPOCHS = 3
    TEST_SPLIT = 0.2
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   # bucket_name = "your-bucket-name"  # Replace with your Google Cloud Storage bucket name
    print(f'Using device: {device}')

    languages = ['hebrew', 'arabic', 'bangla', 'chinese', 'english', 'german', 'indonesian', 'romanian', 'turkish']
    file_paths = {
        'hebrew': '/hebrew.xlsx',
        'arabic': '/arabic.xlsx',
        'bangla': '/bangla.csv',
        'chinese': '/chinese.csv',
        'english': '/english.csv',
        'german': '/german.csv',
        'indonesian': '/indonesian.csv',
        'romanian': '/romanianL.xlsx',
        'turkish': '/turkish.csv'
    }

    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    # Load and clean the Hebrew dataset (for testing)
    df_hebrew = load_data(file_paths['hebrew'])
    df_hebrew = clean_dataset(df_hebrew)

    # Split Hebrew data into training and testing
    df_train_hebrew, df_test_hebrew = train_test_split(df_hebrew, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

    # Loop to train 8 times, excluding one language each time
    for excluded_lang in languages:
        if excluded_lang == 'hebrew':
            continue  # We are not excluding Hebrew, it's used for testing

        print(f"Training model excluding: {excluded_lang}")

        # Combine datasets from all languages except the excluded one
        train_dfs = [df_train_hebrew]
        for lang in languages:
            if lang != excluded_lang and lang != 'hebrew':
                df = load_data(file_paths[lang])
                df = clean_dataset(df)
                train_dfs.append(df)

        # Combine all training data
        df_train_combined = pd.concat(train_dfs, ignore_index=True)

        # Create DataLoader for training and testing
        train_data_loader = create_data_loader(
            df_train_combined['title'].tolist(),
            df_train_combined['label'].tolist(),
            tokenizer,
            MAX_LEN,
            BATCH_SIZE
        )

        test_data_loader_hebrew = create_data_loader(
            df_test_hebrew['title'].tolist(),
            df_test_hebrew['label'].tolist(),
            tokenizer,
            MAX_LEN,
            BATCH_SIZE
        )

        # Initialize the model
        model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
        model = model.to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
        total_steps = len(train_data_loader) * EPOCHS
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
        loss_fn = torch.nn.CrossEntropyLoss().to(device)

        # Training loop
        for epoch in range(EPOCHS):
            print(f'Starting epoch {epoch + 1}/{EPOCHS}')
            train_acc, train_loss = train_epoch(
                model,
                train_data_loader,
                loss_fn,
                optimizer,
                device,
                scheduler,
                len(df_train_combined)
            )
            print(f'Train loss {train_loss}, accuracy {train_acc}')

        # Evaluate on the Hebrew test set
        print(f'Evaluating on Hebrew test set...')
        test_acc, test_loss, labels, preds = eval_model(
            model,
            test_data_loader_hebrew,
            loss_fn,
            device,
            len(df_test_hebrew)
        )
        precision = precision_score(labels, preds, average='weighted')
        recall = recall_score(labels, preds, average='weighted')
        f1 = f1_score(labels, preds, average='weighted')

        print(f'Test Accuracy: {test_acc}')
        print(f'Test Precision: {precision}')
        print(f'Test Recall: {recall}')
        print(f'Test F1: {f1}')

        # Save the trained model to GCS
        model_filename = f"bert_model_excluding_{excluded_lang}.pth"
       # save_model_to_gcs(model, bucket_name, model_filename)

if __name__ == "__main__":
    main()


Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Training model excluding: arabic


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-3-682e2fac421a>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Starting epoch 1/3
Train loss 0.14669799948655596, accuracy 0.938279301745636
Starting epoch 2/3
Train loss 0.0885182198068112, accuracy 0.9638266969937793
Starting epoch 3/3
Train loss 0.05245648598935557, accuracy 0.9795086459674989
Evaluating on Hebrew test set...
Test Accuracy: 0.8578199052132702
Test Precision: 0.857961301665071
Test Recall: 0.8578199052132701
Test F1: 0.8578135179673229
Training model excluding: bangla


...

Starting epoch 1/3
Train loss 0.14218489176923493, accuracy 0.941503455434456
Starting epoch 2/3
Train loss 0.08237388510862123, accuracy 0.9672143461990221
Starting epoch 3/3
Train loss 0.05303919568060689, accuracy 0.9786118167663689
Evaluating on Hebrew test set...
Test Accuracy: 0.8341232227488152
Test Precision: 0.8356521374886304
Test Recall: 0.8341232227488151
Test F1: 0.8339666121015284
Training model excluding: romanian


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-3-682e2fac421a>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Starting epoch 1/3
Train loss 0.15940602812644888, accuracy 0.9321440668397877
Starting epoch 2/3
Train loss 0.1008417455350309, accuracy 0.9586701113775079
Starting epoch 3/3
Train loss 0.06872377068983439, accuracy 0.9720393972198792
Evaluating on Hebrew test set...
Test Accuracy: 0.8246445497630333
Test Precision: 0.8249308527226901
Test Recall: 0.8246445497630331
Test F1: 0.8246209153604135
Training model excluding: turkish


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-3-682e2fac421a>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Starting epoch 1/3
Train loss 0.17953420400624007, accuracy 0.9224026858941005
Starting epoch 2/3
Train loss 0.11074631710332157, accuracy 0.9543967888203375
Starting epoch 3/3
Train loss 0.07474985972121385, accuracy 0.9698580242325132
Evaluating on Hebrew test set...
Test Accuracy: 0.8436018957345972
Test Precision: 0.8444102704109273
Test Recall: 0.8436018957345972
Test F1: 0.843531612348791


In [9]:
#all langs except arabic + predictions
import torch
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to load and clean dataset
def load_data(filepath):
    if filepath.endswith('.csv'):
        return pd.read_csv(filepath)
    elif filepath.endswith('.xlsx'):
        return pd.read_excel(filepath)
    else:
        raise ValueError(f"Unsupported file format: {filepath}")

def clean_dataset(df):
    if 'title' not in df.columns or 'label' not in df.columns:
        raise ValueError("The dataframe must have 'title' and 'label' columns.")
    df = df.dropna(subset=['title', 'label'])  # Ensure no NaNs in title or label
    df = df[df['title'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]  # Ensure non-empty strings
    df = df[df['label'].apply(lambda x: isinstance(x, (int, np.integer)) or str(x).isdigit())]  # Ensure valid labels

    df['title'] = df['title'].astype(str)
    df['label'] = df['label'].astype(int)
    return df

# Dataset wrapper to handle input data
class DatasetWrapper(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def create_data_loader(texts, labels, tokenizer, max_len, batch_size):
    encoding = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )
    dataset = DatasetWrapper(encoding, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training and evaluation functions
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, dataset_len):
    model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / dataset_len, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, dataset_len):
    model.eval()
    losses = []
    correct_predictions = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    return correct_predictions.double() / dataset_len, np.mean(losses), all_labels, all_preds

# Main function to train on all languages except Arabic and test on Hebrew
def main():
    RANDOM_SEED = 42
    MAX_LEN = 128
    BATCH_SIZE = 16
    EPOCHS = 3
    TEST_SPLIT = 0.2
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'Using device: {device}')

    languages = ['hebrew', 'arabic', 'bangla', 'chinese', 'english', 'german', 'indonesian', 'romanian', 'turkish']
    file_paths = {
        'hebrew': '/hebrew.xlsx',
        'arabic': '/arabic.xlsx',
        'bangla': '/bangla.csv',
        'chinese': '/chinese.csv',
        'english': '/english.csv',
        'german': '/german.csv',
        'indonesian': '/indonesian.csv',
        'romanian': '/romanianL.xlsx',
        'turkish': '/turkish.csv'
    }

    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    # Load and clean the Hebrew dataset (for testing)
    df_hebrew = load_data(file_paths['hebrew'])
    df_hebrew = clean_dataset(df_hebrew)

    # Split Hebrew data into training and testing
    df_train_hebrew, df_test_hebrew = train_test_split(df_hebrew, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

    # Combine datasets from all languages except Arabic
    train_dfs = [df_train_hebrew]  # Hebrew is included in the training set
    for lang in languages:
        if lang != 'arabic' and lang != 'hebrew':
            df = load_data(file_paths[lang])
            df = clean_dataset(df)
            train_dfs.append(df)

    # Combine all training data
    df_train_combined = pd.concat(train_dfs, ignore_index=True)

    # Create DataLoader for training and testing
    train_data_loader = create_data_loader(
        df_train_combined['title'].tolist(),
        df_train_combined['label'].tolist(),
        tokenizer,
        MAX_LEN,
        BATCH_SIZE
    )

    test_data_loader_hebrew = create_data_loader(
        df_test_hebrew['title'].tolist(),
        df_test_hebrew['label'].tolist(),
        tokenizer,
        MAX_LEN,
        BATCH_SIZE
    )

    # Initialize the model
    model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
    model = model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
    total_steps = len(train_data_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    loss_fn = torch.nn.CrossEntropyLoss().to(device)

    # Training loop
    for epoch in range(EPOCHS):
        print(f'Starting epoch {epoch + 1}/{EPOCHS}')
        train_acc, train_loss = train_epoch(
            model,
            train_data_loader,
            loss_fn,
            optimizer,
            device,
            scheduler,
            len(df_train_combined)
        )
        print(f'Train loss {train_loss}, accuracy {train_acc}')

    # Evaluate on the Hebrew test set
    print(f'Evaluating on Hebrew test set...')
    test_acc, test_loss, labels, preds = eval_model(
        model,
        test_data_loader_hebrew,
        loss_fn,
        device,
        len(df_test_hebrew)
    )

    # Modify to use the correct order of labels and predictions in the batch
    for i in range(len(labels)):
        title = df_test_hebrew['title'].iloc[i]  # This indexing should be reconsidered
        true_label = labels[i]  # True labels from evaluation
        predicted_label = preds[i]  # Predicted labels from evaluation
        correctness = "טוב" if true_label == predicted_label else "רע"
        print(title)
     #if true_label != predicted_label:
        print(f"Title: {title}")
        print(f"True label: {true_label}, Predicted label: {predicted_label}, {correctness}")
        print("")


    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    print(f'Test Accuracy: {test_acc}')
    print(f'Test Precision: {precision}')
    print(f'Test Recall: {recall}')
    print(f'Test F1: {f1}')

if __name__ == "__main__":
    main()


Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting epoch 1/3


<ipython-input-9-7436542dc634>:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Train loss 0.14683798568623488, accuracy 0.9385944479460689
Starting epoch 2/3
Train loss 0.08883747794400945, accuracy 0.964045929133211
Starting epoch 3/3
Train loss 0.053359373515291394, accuracy 0.9786865254446302
Evaluating on Hebrew test set...
 אל תשקיעו במערכת חימום גדולה עד שתראו את ההמצאה המהפכנית הזו...
Title:  אל תשקיעו במערכת חימום גדולה עד שתראו את ההמצאה המהפכנית הזו...
True label: 0, Predicted label: 0, טוב

 ארגז כלים לטיפול בכאבי גב תחתון באופן עצמאי
Title:  ארגז כלים לטיפול בכאבי גב תחתון באופן עצמאי
True label: 1, Predicted label: 1, טוב

  עינב בובליל: "זה המוצר שגמל אותי ממתוקים"
Title:   עינב בובליל: "זה המוצר שגמל אותי ממתוקים"
True label: 1, Predicted label: 1, טוב

אלעל פתחה עמדות צ'ק אין עצמי בנתב"ג
Title: אלעל פתחה עמדות צ'ק אין עצמי בנתב"ג
True label: 1, Predicted label: 1, טוב

הטעות של בני 55+
Title: הטעות של בני 55+
True label: 1, Predicted label: 0, רע

 מיכאל בן דוד: 'כעסתי על אמא שלי שלא הגיעה לחתונה שלי'
Title:  מיכאל בן דוד: 'כעסתי על אמא שלי שלא הג

In [8]:
for i in range(len(labels)):
        title = df_test_hebrew['title'].iloc[i]  # This indexing should be reconsidered
        true_label = labels[i]  # True labels from evaluation
        predicted_label = preds[i]  # Predicted labels from evaluation
        correctness = "טוב" if true_label == predicted_label else "רע"
        print(true_label)
   # if true_label != predicted_label:
   #     print(f"Title: {title}")
   #     print(f"True label: {true_label}, Predicted label: {predicted_label}, {correctness}")
   #     print("")

NameError: name 'labels' is not defined

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to load and clean dataset
def load_data(filepath):
    if filepath.endswith('.csv'):
        return pd.read_csv(filepath)
    elif filepath.endswith('.xlsx'):
        return pd.read_excel(filepath)
    else:
        raise ValueError(f"Unsupported file format: {filepath}")

def clean_dataset(df):
    if 'title' not in df.columns or 'label' not in df.columns:
        raise ValueError("The dataframe must have 'title' and 'label' columns.")
    df = df.dropna(subset=['title', 'label'])  # Ensure no NaNs in title or label
    df = df[df['title'].apply(lambda x: isinstance(x, str) and len(x.strip()) > 0)]  # Ensure non-empty strings
    df = df[df['label'].apply(lambda x: isinstance(x, (int, np.integer)) or str(x).isdigit())]  # Ensure valid labels

    df['title'] = df['title'].astype(str)
    df['label'] = df['label'].astype(int)
    return df

# Dataset wrapper to handle input data
class DatasetWrapper(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def create_data_loader(texts, labels, tokenizer, max_len, batch_size):
    encoding = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )
    dataset = DatasetWrapper(encoding, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training and evaluation functions
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, dataset_len):
    model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        _, preds = torch.max(outputs.logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / dataset_len, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, dataset_len):
    model.eval()
    losses = []
    correct_predictions = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    return correct_predictions.double() / dataset_len, np.mean(losses), all_labels, all_preds

# Main function to train and test models
def main():
    RANDOM_SEED = 42
    MAX_LEN = 128
    BATCH_SIZE = 16
    EPOCHS = 3
    TEST_SPLIT = 0.2
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'Using device: {device}')

    languages = ['hebrew', 'arabic', 'bangla', 'chinese', 'english', 'german', 'indonesian', 'romanian']
    file_paths = {
        'hebrew': '/hebrew.xlsx',
        'arabic': '/arabic.xlsx',
        'bangla': '/bangla.csv',
        'chinese': '/chinese.csv',
        'english': '/english.csv',
        'german': '/german.csv',
        'indonesian': '/indonesian.csv',
        'romanian': '/romanianL.xlsx'
    }

    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

    # Load and clean datasets for all languages
    dfs = {lang: clean_dataset(load_data(file_paths[lang])) for lang in languages}

    # Initialize a list to store the results
    results = []

    # Loop to train on each language and test on all others
    for train_lang in languages:
        print(f"\nTraining on {train_lang}")
        df_train = dfs[train_lang]

        # Train/Test split for the training language
        df_train_lang, df_val_lang = train_test_split(df_train, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

        # Create DataLoader for training
        train_data_loader = create_data_loader(
            df_train_lang['title'].tolist(),
            df_train_lang['label'].tolist(),
            tokenizer,
            MAX_LEN,
            BATCH_SIZE
        )

        val_data_loader = create_data_loader(
            df_val_lang['title'].tolist(),
            df_val_lang['label'].tolist(),
            tokenizer,
            MAX_LEN,
            BATCH_SIZE
        )

        # Initialize the model
        model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
        model = model.to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
        total_steps = len(train_data_loader) * EPOCHS
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
        loss_fn = torch.nn.CrossEntropyLoss().to(device)

        # Training loop
        for epoch in range(EPOCHS):
            print(f'Epoch {epoch + 1}/{EPOCHS}')
            train_acc, train_loss = train_epoch(
                model,
                train_data_loader,
                loss_fn,
                optimizer,
                device,
                scheduler,
                len(df_train_lang)
            )
            print(f'Train loss: {train_loss}, accuracy: {train_acc}')

        # Evaluate on all other languages
        for test_lang in languages:
            if test_lang == train_lang:
                continue  # Skip the training language for testing

            print(f"\nEvaluating on {test_lang}")
            df_test = dfs[test_lang]

            test_data_loader = create_data_loader(
                df_test['title'].tolist(),
                df_test['label'].tolist(),
                tokenizer,
                MAX_LEN,
                BATCH_SIZE
            )

            test_acc, test_loss, labels, preds = eval_model(
                model,
                test_data_loader,
                loss_fn,
                device,
                len(df_test)
            )

            precision = precision_score(labels, preds, average='weighted')
            recall = recall_score(labels, preds, average='weighted')
            f1 = f1_score(labels, preds, average='weighted')

            # Store the result
            results.append({
                'Source Language': train_lang,
                'Target Language': test_lang,
                'Accuracy': test_acc.item(),  # Convert to float for compatibility with DataFrame
                'Precision': precision,
                'Recall': recall,
                'F1-Score': f1
            })

    # Convert the results to a DataFrame and print as a table
    results_df = pd.DataFrame(results)
    print("\nCross-Language Results:")
    print(results_df)

if __name__ == "__main__":
    main()


Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



Training on hebrew


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


...


Evaluating on german


<ipython-input-2-590ea965ac34>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}



Evaluating on indonesian


<ipython-input-2-590ea965ac34>:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}



Cross-Language Results:
   Source Language Target Language  Accuracy  Precision    Recall  F1-Score
0           hebrew          arabic  0.514995   0.527476  0.514995  0.452858
1           hebrew          bangla  0.670539   0.732278  0.670539  0.677997
2           hebrew         chinese  0.822226   0.822998  0.822226  0.804932
3           hebrew         english  0.890062   0.893807  0.890062  0.889801
4           hebrew          german  0.457678   0.664310  0.457678  0.538242
5           hebrew      indonesian  0.691533   0.712008  0.691533  0.663056
6           hebrew        romanian  0.728720   0.727873  0.728720  0.727772
7           arabic          hebrew  0.411989   0.411209  0.411989  0.408571
8           arabic          bangla  0.451452   0.521020  0.451452  0.460844
9           arabic         chinese  0.467219   0.570224  0.467219  0.492709
10          arabic         english  0.446844   0.442482  0.446844  0.436135
11          arabic          german  0.392155   0.730122  0.3921

In [ ]:
gcloud storage buckets list


SyntaxError: invalid syntax (<ipython-input-10-277ce96ec27d>, line 1)